In [1]:
########################################################################################################################
# This script extracts the needed potential predictors as patient-level features
# Step 1. Identify the needed potential predictors
# Step 2. Identify the needed rows (i.e., patients to be considered)
# Step 3. Retain only variables with adequate records
# Step 4. Split into training and test set using the target data created in script P02_Stratified_Partitioning
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import warnings
from itertools import product
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
####################################################################################################################
# USER_SPECIFIC SETTING
# PAT_PATH: Path of the patient-level dataset (created in C07_Date_Adjustment.ipynb)
# TARGET_DIR_PATH: Path of the directory storing the target datasets (created in P01_Subject_Inclusion.ipynb)
# OUT_DIR_PATH: Path of the directory of the output feature datasets
####################################################################################################################
PAT_PATH: str = '../00_Data/01_Cleaned_Data/Patient_full_v2.parquet'
TARGET_DIR_PATH: str = '../00_Data/02_Processed_Data/Targets/'
OUT_FEAT_DIR_PATH: str = '../00_Data/02_Processed_Data/Features/'

In [ ]:
########################################################################################################################
# USER_SPECIFIC SETTING
# NAN_THRESHOLD: A float p between 0 and 1 such that variables with less than (p * 100)% records will not be included
########################################################################################################################
NAN_THRESHOLD: float = 0.35

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [1, 2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [4]:
####################################################################################################################
# Load the patient-level dataset
####################################################################################################################
df_pat: pd.DataFrame = pd.read_parquet(PAT_PATH)
df_pat.drop(columns='BirthDate', inplace=True, errors='ignore')
pat_cols: list[str] = df_pat.columns.tolist()
pat_cols = [c for c in pat_cols if c != 'PatientDurableKey']
print(f'Patient-level data loaded with dimension {df_pat.shape}')

Patient-level data loaded with dimension (578521, 307)


In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds
########################################################################################################################
for exp_idx, (C, D) in enumerate(product(Cs, Ds), 1):
    log_head: str = f'[{exp_idx}. C={C}; D={D}] '
    
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the target file created in P01_Subject_Inclusion
    ####################################################################################################################
    y_path: str = os.path.join(TARGET_DIR_PATH, f'{C}_encounters_{D}_days_v1.csv')
    df_y: pd.DataFrame = pd.read_csv(y_path)

    ####################################################################################################################
    # Identify the patients to be included
    ####################################################################################################################
    y_pats: list[str] = df_y['PatientDurableKey'].unique()
    assert len(y_pats) == df_y.shape[0]
    print(f'Extracting {len(y_pats)} patients from the patient-level dataset.')

    ####################################################################################################################
    # Extract the needed patients
    ####################################################################################################################
    df_pat_cur: pd.DataFrame = df_pat[df_pat['PatientDurableKey'].isin(y_pats)]

    ####################################################################################################################
    # Retain only columns with at least (1 - NAN_THRESHOLD)% data
    ####################################################################################################################
    N: int = df_pat_cur.shape[0]
    remove_cols: list[str] = [col for col in pat_cols if df_pat_cur[col].isna().sum() >= N * NAN_THRESHOLD]
    print(f'{log_head}{len(remove_cols)} (out of {len(pat_cols)}) have >={int(NAN_THRESHOLD*100)}% missing values and will be removed.')
    df_pat_cur.drop(columns=remove_cols, inplace=True)
    print(f'{log_head}Updated patient-level data has a dimension of {df_pat_cur.shape} (with 1 indexing column).')

    ####################################################################################################################
    # Load the partitions of y separately
    ####################################################################################################################
    for partition in ['train', 'test']:
        y_part_path: str = y_path.replace('_v1.csv', f'_{partition}_v2.parquet')
        df_y_part: pd.DataFrame = pd.read_parquet(y_part_path)[['PatientDurableKey']].copy()
        df_pat_cur_part: pd.DataFrame = pd.merge(left=df_pat_cur, right=df_y_part, on='PatientDurableKey', how='right')
        
        ################################################################################################################
        # Save the patient-level feature dataset
        ################################################################################################################
        out_dir_path_s: str = os.path.join(OUT_FEAT_DIR_PATH, f'{C}_encounters_{D}_days/')
        os.makedirs(out_dir_path_s, exist_ok=True)
        out_file_path: str = f'{out_dir_path_s}X_Patient_{partition}_v1.parquet'
        df_pat_cur_part.to_parquet(out_file_path)
        print(f'{log_head} ({partition}) Patient-level dataset saved with dimension = {df_pat_cur_part.shape}')
    print('-'*120)